In [71]:
import requests
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

path_phantom = '//Applications/phantomjs'

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

from wordcloud import WordCloud, ImageColorGenerator, get_single_color_func

In [72]:
cities=pd.read_csv("data/Living_Index.csv")
nyse=pd.read_csv("data/NYSE.csv")
nasdaq=pd.read_csv("data/NASDAQ.csv")
amex=pd.read_csv("data/AMEX.csv")
stocks=nyse.append(nasdaq,ignore_index=True).append(amex,ignore_index=True)
cities=cities.rename(columns={"Urban Area":"City"})
city_list=[x for x in cities['City']]

In [73]:
salaries = pd.read_json("data/allcities.json")

In [74]:
len(salaries)

1546

In [75]:
def only_numerics(seq):
    seq = seq.split('-')
    try:
        out = ((int(filter(type(seq[0]).isdigit, seq[0])) + int(filter(type(seq[1]).isdigit, seq[1])))/2)*1000
    except:
        out = int(filter(type(seq[0]).isdigit, seq[0]))
    return out

salaries['meanPay'] = salaries['meanPay'].apply(only_numerics)

In [76]:
salaries = salaries[salaries.jobTitle.str.contains("Intern") == False]
salaries = salaries[salaries.jobTitle.str.contains("Hourly") == False]
salaries['jobTitle'].unique()

array([u'Data Scientist', u'Entry Level Data Scientist',
       u'Senior Data Scientist', u'Junior Data Scientist',
       u'Principal Data Scientist', u'Data Scientist II',
       u'Data Scientist I', u'Associate Data Scientist',
       u'Chief Data Scientist',
       u'Clinical Laboratory Scientist-data Analyst',
       u'Senior Data Scientist/Statistician',
       u'Scientist, Statistical and Data Sciences',
       u'Data Visualization Scientist', u'Staff Data Scientist',
       u'Lead Data Scientist', u'Software Engineer (Data Scientist)'], dtype=object)

In [77]:
salaries['state'] = salaries.city.apply(lambda x: x[-2:])

Remove duplicates:

In [78]:
cities = salaries.groupby(['city', 'state'])['city'].count().to_frame()

In [79]:
cities.rename(columns={'city': 'count'}, inplace=True)
cities.reset_index(inplace=True)

In [80]:
cities['duplicated'] = cities.duplicated(['count', 'state'], keep=False)

In [81]:
cities[cities['duplicated']==True].sort_values(by=['state','count'])

,city,state,count,duplicated
55,"Oakland, CA",CA,134,True
75,"San Francisco, CA",CA,134,True
9,"Bradenton, FL",FL,1,True
78,"Sarasota, FL",FL,1,True
2,"Atlanta, GA",GA,42,True
48,"Marietta, GA",GA,42,True
6,"Baton Rouge, LA",LA,2,True
36,"Hammond, LA",LA,2,True
45,"Lafayette, LA",LA,2,True
50,"Minneapolis, MN",MN,11,True


In [82]:
# What to remove:
removables = ['Everett, WA', 'Tacoma, WA', 'Weatherford, TX', 'Plano, TX', 'Fort Worth, TX', 'Arlington, TX', 'San Marcos, TX', 'Round Rock, TX', 'Conroe, TX', 'Seguin, TX', 'Morristown, TN', 'Sumter, SC', 'Camden, SC', 'Durham, NC', 'Chapel Hill, NC', 'St. Paul, MN', 'Hammond, LA', 'Marietta, GA', 'Bradenton, FL', 'Oakland, CA']

In [83]:
salaries = salaries[~salaries['city'].isin(removables)]
salaries.reset_index(inplace=True)
del salaries['index']

Remove state name from city

In [84]:
salaries['city'] = salaries.city.apply(lambda x: x[:-4])

Rearrange state to be next to city

In [90]:
cols = ['city', 'state', 'company', 'jobTitle', 'meanPay']
salaries = salaries[cols]
salaries.company.unique()

array([u'GE', u'CareerBuilder', u'NCR', u'The Home Depot', u'Feedzai',
       u'IBM', u'Booz Allen Hamilton', u'PwC',
       u'Cognizant Technology Solutions', u'Rang Technologies', u'KPMG',
       u'Gallup', u'Hewlett Packard Enterprise | HPE', u'Verizon',
       u'Verizon Wireless', u'Fractal Analytics', u'Equifax',
       u'Northrop Grumman', u'ADP', u'Kabbage', u'Citi', u'Honeywell',
       u'Intuitive Surgical', u'AT&T', u'Cox Enterprises', u'Hearst',
       u"Macy's", u'Bank of America', u'Exponent', u'Razorfish',
       u'Market Strategies International', u'Experian', u'Think Big',
       u'Square Root', u'Adometry', u'Vast', u'Facebook', u'Apple',
       u'Amazon.com', u'eBay', u'Visa Inc.', u'NTT DATA', u'Zynga',
       u'General Motors', u'The Advisory Board Company',
       u'Clockwork Solutions', u'Intel Corporation', u'MaxPoint', u'Dell',
       u'Cisco Systems', u'Philips', u'MITRE', u'CORMAC', u'Nielsen',
       u'US Department of Defense', u'General Dynamics Mission Sys

In [86]:
salaries[salaries['city'] == 'Cincinnati']

,city,state,company,jobTitle,meanPay
214,Cincinnati,OH,84.51°,Data Scientist,102000
215,Cincinnati,OH,Booz Allen Hamilton,Data Scientist,73000
216,Cincinnati,OH,GE,Data Scientist,110000
217,Cincinnati,OH,Vantiv,Data Scientist,88000
218,Cincinnati,OH,GE,Senior Data Scientist,124000


# Getting job descriptions from Indeed

In [20]:
## YOUR CODE HERE
def extract_location_from_result(result):
    try:
        location = result.find('span', class_='location').text.strip()
    except:
        location = ''
    return location

def extract_title_from_result(result):
    try:
        job_title = result.find('a', class_='jobtitle').text.strip('\n')
    except:
        try:
            job_title = result.find('h2', class_='jobtitle').text.strip('\n')
        except:
            job_title = ''
    return job_title

def extract_summary_from_result(result):
    try:
        summary = result.find('span', class_='summary').text.strip()
    except:
        summary = ''
    return summary

def extract_company_from_result(result):
    try:
        company = result.find('span', class_='company').text.strip()
    except:
        company = ''
    return company

def extract_reviews_from_result(result):
    try:
        sub = result.find('span', class_='slNoUnderline').text.strip()
        if 'review' in sub:
            reviews = sub.split()[0]
    except:
        reviews = ''
    return reviews

def extract_salary_from_result(result):
    try:
        salary = result.find('td', class_='snip').find('nobr').text.strip()
#         salary_text = salary_text.split()
#         salary = float(salary_text[0].strip('$').replace(',',''))
    except:
        salary = ''
    return salary

def get_all_details(result):
#     if extract_salary_from_result(result) != '':
    return [extract_title_from_result(result),\
            extract_company_from_result(result),\
            extract_location_from_result(result),\
            extract_summary_from_result(result)]
           # extract_salary_from_result(result)]
#     else:
#         return 'skip'

In [88]:
search_list = [['+'.join(x[0].split(' ')), x[1], '+'.join((x[3] + ' ' + x[2]).split(' '))] for x in \
              zip(salaries['city'], salaries['state'], salaries['company'],salaries['jobTitle'])]

In [101]:
for i in range(len(search_list)):
    if '84.51' in search_list[i][2]:
        search_list[i][2] = search_list[i][2][:-1]

In [105]:
url_template = 'http://www.indeed.com/jobs?q={}&l={}%2C+{}'
results = []
driver = webdriver.PhantomJS(executable_path=path_phantom)

In [106]:
for i in search_list:
    full_url = url_template.format(i[2], i[1], i[0])
    driver.get(full_url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # Append to the full set of results
    out = get_all_details(soup.find('div', class_='result'))
    out.extend(i)
    results.append(out)
#         pass

In [111]:
indeed_df = pd.DataFrame(results)
indeed_df.head()

,0,1,2,3,4,5,6,7
0,Bioinformatics Data Scientist,GE Corporate,"Niskayuna, NY","As a Data Scientist, you will:. Work with a te...",,Albany,NY,Data+Scientist+GE
1,Data Scientist (Data Science R&D),CareerBuilder,"Norcross, GA 30092",Data Scientist (Data Science R&D). The team al...,"$10,000 a year",Atlanta,GA,Data+Scientist+CareerBuilder
2,Data Scientist-1,NCR,"Duluth, GA 30096",NCR (. About NCR Corporation. NCR only accepts...,,Atlanta,GA,Data+Scientist+NCR
3,DATA SCIENTIST,Home Depot,"Atlanta, GA 30354",Strong communication and data presentation ski...,,Atlanta,GA,Data+Scientist+The+Home+Depot
4,Data Scientist,Feedzai,"Atlanta, GA",Feedzai is an Equal Opportunity Employer. If y...,,Atlanta,GA,Data+Scientist+Feedzai


In [112]:
indeed_df.columns = ['title', 'company', 'location', 'description',\
                     'drop', 'search_city', 'search_state', 'search_title_company']

In [120]:
indeed_df.head(20)

,title,company,location,description,drop,search_city,search_state,search_title_company,city,in_city
0,Bioinformatics Data Scientist,GE Corporate,"Niskayuna, NY","As a Data Scientist, you will:. Work with a te...",,Albany,NY,Data+Scientist+GE,Niskayuna,Niskayuna
1,Data Scientist (Data Science R&D),CareerBuilder,"Norcross, GA 30092",Data Scientist (Data Science R&D). The team al...,"$10,000 a year",Atlanta,GA,Data+Scientist+CareerBuilder,Norcross,Norcross
2,Data Scientist-1,NCR,"Duluth, GA 30096",NCR (. About NCR Corporation. NCR only accepts...,,Atlanta,GA,Data+Scientist+NCR,Duluth,Duluth
3,DATA SCIENTIST,Home Depot,"Atlanta, GA 30354",Strong communication and data presentation ski...,,Atlanta,GA,Data+Scientist+The+Home+Depot,Atlanta,Atlanta
4,Data Scientist,Feedzai,"Atlanta, GA",Feedzai is an Equal Opportunity Employer. If y...,,Atlanta,GA,Data+Scientist+Feedzai,Atlanta,Atlanta
5,Software Engineers - Data Analytics Engineer,"Solidyn Solutions, Inc",United States,Certification in Cloudera data technologies (C...,,Atlanta,GA,Data+Scientist+IBM,United States,United States
6,,,,,,Atlanta,GA,Data+Scientist+Booz+Allen+Hamilton,,
7,,,,,,Atlanta,GA,Data+Scientist+PwC,,
8,,,,,,Atlanta,GA,Data+Scientist+Cognizant+Technology+Solutions,,
9,,,,,,Atlanta,GA,Data+Scientist+Rang+Technologies,,


In [121]:
indeed_df['in_city'] = indeed_df['location'].map(lambda x: x.split(',')[0] if x != 'United States' and x != '' else x)

In [124]:
indeed_df['in_state'] = indeed_df['location'].map(lambda x: x.split(',')[1][1:3] if x != 'United States' and x != '' else x)

In [125]:
indeed_df.head()

,title,company,location,description,drop,search_city,search_state,search_title_company,city,in_city,in_state
0,Bioinformatics Data Scientist,GE Corporate,"Niskayuna, NY","As a Data Scientist, you will:. Work with a te...",,Albany,NY,Data+Scientist+GE,Niskayuna,Niskayuna,NY
1,Data Scientist (Data Science R&D),CareerBuilder,"Norcross, GA 30092",Data Scientist (Data Science R&D). The team al...,"$10,000 a year",Atlanta,GA,Data+Scientist+CareerBuilder,Norcross,Norcross,GA
2,Data Scientist-1,NCR,"Duluth, GA 30096",NCR (. About NCR Corporation. NCR only accepts...,,Atlanta,GA,Data+Scientist+NCR,Duluth,Duluth,GA
3,DATA SCIENTIST,Home Depot,"Atlanta, GA 30354",Strong communication and data presentation ski...,,Atlanta,GA,Data+Scientist+The+Home+Depot,Atlanta,Atlanta,GA
4,Data Scientist,Feedzai,"Atlanta, GA",Feedzai is an Equal Opportunity Employer. If y...,,Atlanta,GA,Data+Scientist+Feedzai,Atlanta,Atlanta,GA


In [130]:
indeed_df['city_match'] = (indeed_df['search_city']==indeed_df['in_city'])

In [131]:
indeed_df['state_match'] = ((indeed_df['search_state']==indeed_df['in_state']))

In [135]:
indeed_df['location_match'] = [all(i) for i in zip(indeed_df['city_match'] ,indeed_df['state_match'] )]

In [137]:
indeed_df.location_match.value_counts()

False    825
True     136
Name: location_match, dtype: int64